In [1]:
from cmath import inf
from math import sqrt
import math
import random

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #make tensorflow stop talking

import evaluator
import constants
import chess
import chess.pgn
import btm
import zeromodel
import zero_mcts
from player import Player
import data

import numpy as np
import tensorflow as tf
from keras.models import Model

In [2]:
model = zeromodel.build_zero_model(constants.NUM_RES_LAYERS)
storage = data.Data()

In [4]:
game_num = 1
while True:

    white = Player(chess.Board(), model)
    black = Player(chess.Board(), model)
    board = chess.Board()

    cur_player = white
    other_player = black

    turn = chess.WHITE
    move_count = 0
    temp = 1
    print('playing game', game_num)
    while not board.is_game_over(claim_draw=True):
        if (turn == chess.WHITE):
            cur_player = white
            other_player = black
        else:
            cur_player = black
            other_player = white
        
        if (move_count >= constants.ND_TURNS and temp != 0):
            temp = 0
        
        move, pi = cur_player.make_move(constants.MCTS_DEPTH, temp)
        other_player.change_root(move)

        move_count += 1
        turn = not turn

        board_array = btm.board_to_array(board)
        storage.add_move(turn, board_array, pi)

        board.push(move)
        print(move)

    outcome = board.outcome(claim_draw=True)
    storage.save_game(outcome)
    game_num += 1

    if len(storage.history) >= constants.MOVE_STORAGE:
        print('training')
        for i in range(constants.TRAINING_LOOPS):
            batch = random.sample(storage.history, constants.TRAIN_SIZE)
            features = np.array([move['board'] for move in batch])
            pi = np.array([move['pi'] for move in batch])
            z = np.array([move['z'] for move in batch])
            labels = {'policy_head': pi, 'value_head': z}

            model.fit(features, labels, epochs=1, verbose=1)
        model.save('zero-model.keras')

playing game 1
f2f3


In [4]:
model.save('zero-model.keras')